In [3]:
import time
from bs4 import BeautifulSoup
import tqdm
import re
import json
from collections import defaultdict
import random
import ssl

## Helper functions

In [1]:
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

def get_soup(url):
    html = requests.get(url, headers={"User-Agent": random.choice(user_agent_list)}).text
    soup = BeautifulSoup(html, "html.parser")
    return soup

def get_companies(url):
    soup = get_soup(url).tbody
    tags = soup('tr')
    companies = [tag.find_all('a')[1].text for tag in tags]
    return companies

## Create SSL Context

In [5]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

## Source Companies and Get URLs for Transcripts

In [ ]:
companies = get_companies("http://www.slickcharts.com/sp500")

In [ ]:
urls = defaultdict(dict)
for company in tqdm.tqdm_notebook(companies):
    url = f"http://www.seekingalpha.com/symbol/{company}/earnings/transcripts"
    soup = get_soup(url)
    divs = soup.find_all("div", class_ = "symbol_article")
    for div in divs:
        if "Earnings Call Transcript" in div.a.text:
            url = "http://www.seekingalpha.com" + div.a.get("href") + "?part=single"
            title = div.a.text
            period = re.search('Q[0-9] [0-9][0-9][0-9][0-9]', title)
            if not period:
                print(f"{company} date not found")
                print(title)
                continue
            urls[company][period] = url
    time.sleep(np.random.random()+4)

In [ ]:
with open('transcript_urls.json', 'w') as out:
    json.dump(new_urls, out, indent=2)

## Start Scraping Actual Transcripts

In [ ]:
with open('transcript_urls.json', 'r') as inp:
    transcript_urls = json.load(inp)

In [ ]:
companies = random.choices(list(transcript_urls.keys()), k=5)
transcripts = []
for company in companies:
    quarters = list(transcript_urls[company].keys())
    quarter = random.choice(quarters)
    transcripts.append({
        'company': company,
        'quarter': quarter,
        'url': transcript_urls[company][quarter]
    })

## Create Collection in Postman

In [ ]:
items = []
for transcript in transcripts:
    item = {
        'name': transcript['company'] + ' ' + transcript['quarter'],
        'protocolProfileBehavior': {'disableBodyPruning': True},
        'request': {'method': 'GET',
            'header': [],
            'url': {
                'raw': transcript['url'],
                'protocol': 'https',
                'host': ['seekingalpha', 'com'],
                'path': ['article'] + transcript['url'].split('/article/')[1].split('?')[0].split('/'),
                'query': [{'key': 'part', 'value': 'single'}]
            }
        },
        'response': []
    }
    items.append(item)

In [ ]:
collection_body = {'collection': {'info': {'_postman_id': 'c5147b08-af9f-4ea3-b8b0-b04984c8a075',
   'name': 'Write Responses To File',
   'description': "This collection makes it easy for users who want to write the response of each request to a file.\nThis can be extended to write anything for eg. meta information or value of variables being used.\n\n\nThis project is using a powerful feature built in postman called as `pm.sendRequest`, the docs for which can be found here: https://www.getpostman.com/docs/v6/postman/scripts/postman_sandbox_api_reference#pmsendrequest\n\n\n## Requirements\nTo work with this script, a local server is required.\nYou can choose to write your own local server or edit the one provided below as per your needs.\n\n## Steps To Use\n1. Put all the requests you want to write the responses for, under this collection.\n \n2. Clone the following repository to your machine - https://github.com/sivcan/ResponseToFile-Postman or use the following command - `git clone https://github.com/sivcan/ResponseToFile-Postman`\n\n3. Navigate into the directory and install the dependencies. Use the following command: `npm i` \n\n4. Run the local server. Use the following command: `node script.js`\n\n5. Now, the responses for every request which is a part of this collection will be written to the `Responses` folder inside the project repo.\nYou can modify the local server's code to change the file location.\n\n5. Run your requests through builder / run through collection runner and store your data locally. \n\n\nYou can modify the script and the local server to support more data formats / data that you want to write to the file.\n\n## Additionally\nInstead of moving each request under this collection, you can copy the script from the `Tests` tab of this collection to the `Tests` tab of any request or even a specific folder.\n\n**Note:** To access the `Tests` script of this collection:\n1. You need to `Right Click` the `Write Responses To File` collection in the sidebar.\n2. Click on `Edit`\n3. Go to `Tests` tab.\n\nThen you can send that particular request / requests under a folder for which the data needs to be written.\n\n## File Extensions\nYou can also modify the **extension** of the file.\n\n**Example:**\nIncase you want to write CSV data to a file, all you need to do is change the `fileExtension` property in the `Tests` script to `csv`.",
   'schema': 'https://schema.getpostman.com/json/collection/v2.1.0/collection.json'},
  'item': items,
  'event': [{'listen': 'prerequest',
    'script': {'id': '046bd3de-4c08-4521-8aaa-f94ebb58c57e',
     'type': 'text/javascript',
     'exec': ['']}},
   {'listen': 'test',
    'script': {'id': 'ad202c49-0ed3-41c5-a627-80da06478953',
     'type': 'text/javascript',
     'exec': ['// The data to be written to file',
      'let dataToFile = {',
      '    requestName: request.name || request.url,',
      "    fileExtension: 'json',",
      '    responseData: pm.response.text()',
      '};',
      '',
      'pm.sendRequest({',
      "    url: 'http://localhost:3000/write',",
      "    method: 'POST',",
      "    header: 'Content-Type:application/json',",
      '    body: {',
      "        mode: 'raw',",
      '        raw: JSON.stringify(dataToFile)',
      '    }',
      '}, function(err, res) {',
      '    console.log(res);',
      '});']}}]}}

In [ ]:
def get(url):
    return requests.get(url, headers={'X-Api-Key': api_key})
def put(url, body):
    return requests.put(url, headers={'X-Api-Key': api_key, 'Content-Type': 'application/json'}, data=body)

In [ ]:
api_key = '<PASTE YOUR API KEY HERE>'
r = get('https://api.getpostman.com/collections').json()
earnings_collection_id = r['collections'][0]['id']

## Push new collection to Postman

In [ ]:
r = put(f'https://api.getpostman.com/collections/{earnings_collection_id}', json.dumps(collection_body))